In [13]:
from config import DATABASE_URI
import psycopg2
from PIL import Image
import io
import numpy as np
import random

In [14]:
def get_images_from_db():
    # Connect to the database
    conn = psycopg2.connect(DATABASE_URI)

    cursor = conn.cursor()
    
    # Query to get all fruit images
    cursor.execute("SELECT fruit_name, image FROM fruits")
    rows = cursor.fetchall()

    images = []
    labels = []
    for row in rows:
        fruit_name, binary_data = row
        image = Image.open(io.BytesIO(binary_data))
        image = image.resize((128, 128))  # Resize for consistency
        images.append(np.array(image))
        labels.append(fruit_name)
    
    cursor.close()
    conn.close()
    
    return np.array(images), np.array(labels)

images, labels = get_images_from_db()
print(f'Retrieved {len(images)} images from the database.')

Retrieved 9678 images from the database.


In [15]:
# Normalize the images
images = images / 255.0

# Encode labels as integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [17]:

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [23]:
# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\Javier Alarcon\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,267,973 (12.47 MB)

 Trainable params: 3,267,973 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 34s 122ms/step - accuracy: 0.3074 - loss: 1.6220 - val_accuracy: 0.5424 - val_loss: 1.1064
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 116ms/step - accuracy: 0.5344 - loss: 1.1127 - val_accuracy: 0.5847 - val_loss: 1.0261
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 117ms/step - accuracy: 0.5864 - loss: 1.0060 - val_accuracy: 0.6358 - val_loss: 0.9481
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 117ms/step - accuracy: 0.6449 - loss: 0.8761 - val_accuracy: 0.6302 - val_loss: 0.9188
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 119ms/step - accuracy: 0.7116 - loss: 0.7463 - val_accuracy: 0.6389 - val_loss: 0.9116
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 120ms/step - accuracy: 0.7827 - loss: 0.5925 - val_accuracy: 0.6493 - val_loss: 0.9641
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 118ms/step - accuracy: 0.8745 - loss: 0.3570 - val_accuracy: 0.6550 - val_loss: 1.1787
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 118ms/step - accuracy: 0.9377 - loss: 0

In [24]:
# Save the model
model.save('NeuralNetwork_train_model/fruit_classifier_model_two.h5')

In [21]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

61/61 - 2s - 28ms/step - accuracy: 0.6209 - loss: 1.9515
Test accuracy: 0.6208677887916565


In [25]:
# Predict on test data
predictions = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step


In [26]:
# Print some of the predictions to see how the model performs
for i in range(50):
    print(f'Actual: {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted: {predicted_labels[i]}')

Actual: Strawberry, Predicted: Mango
Actual: Banana, Predicted: Mango
Actual: Mango, Predicted: Mango
Actual: Strawberry, Predicted: Strawberry
Actual: Apple, Predicted: Strawberry
Actual: Banana, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Grape, Predicted: Strawberry
Actual: Grape, Predicted: Banana
Actual: Grape, Predicted: Strawberry
Actual: Apple, Predicted: Banana
Actual: Strawberry, Predicted: Strawberry
Actual: Apple, Predicted: Banana
Actual: Mango, Predicted: Mango
Actual: Mango, Predicted: Banana
Actual: Banana, Predicted: Banana
Actual: Mango, Predicted: Banana
Actual: Grape, Predicted: Banana
Actual: Apple, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Strawberry, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Strawberry, Predicted: Banana
Actual: Strawberry, Predicted: Banana
Actual: Apple, Predicted: Strawberry
Actual: Banana, Predicted: Banana
Actual: Banana, Predicted: Banana
Actual: Strawberry, Predicted: Banana
Actu

In [27]:
# Save the label encoder
import pickle

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)